# Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
# from pandas_profiling import ProfileReport
import numpy as np
import ast
from ast import literal_eval
from dateutil.parser import parse

# Functions

In [2]:
def lowerCase(df):
    '''
    This function changes the names of column and the columns values that are string, into lowercase
    '''
    # converts the name of columns to lowercase
    df.columns = df.columns.str.lower()

    # converts the string values of columns to lowercase
    for column in list(df.columns):
        if df[column].dtype == 'object':
            df[column] = df[column].map(
                lambda x: x.lower() if isinstance(x, str) else x)
        else:
            continue
    
    return df

In [3]:
def strip(df):
    '''
    This function removes the white space at the beginning and end of string values
    '''
    for c in df.columns:
        # it is only applied to columns of the type object
        if df[c].dtype == 'object':
            df[c] = df[c].str.strip()
            df[c] = df[c].str.strip('-')
        else:
            pass
    
    return df

In [4]:
def create_ID(df, cList, cName, idName):
    df2 = df[cList]
    df2 = df2.dropna(how='all')
    df2 = df2.drop_duplicates(keep='last').reset_index()
    df2 = df2.drop(labels='index', axis=1)

    df2[cName] = df2.index + 1
    df2[cName] = df2[cName].astype(str)
    df2[cName] = idName + df2[cName]

    return df2

In [5]:
def remove(column,df):
    '''
    This function removes the duplicated columns based on the number of null values
        e.g. between genre_x and genre_y, the one that has less null values will be removed
    column: A list of columns
    '''

    # Duplicated columns are used to fill the null values of each other
    # e.g. genre_x is used to fill null values of genre_y and vise versa
    for c in column:
        x = c + '_x'
        y = c + '_y'
        if x in list(df.columns):
            df[x] = df[x].fillna(df[y])
            df[y] = df[y].fillna(df[x])

    # Between duplicated columns, the one with more null values will be removed
    for c in column:
        x = c + '_x'
        y = c + '_y'
        if x in list(df.columns):
            if df[x].isna().sum() <= df[y].isna().sum():
                df = df.drop(labels=y, axis=1)
                df.rename(columns={x:c}, inplace=True)
            else:
                df = df.drop(labels=x, axis=1)
                df.rename(columns={y:c}, inplace=True)

    return df 

# Importing data

The data which is the result of preprocessing is imported here so we can do some additional operations such as combining tables and adding IDs.

In [6]:
final_anime_1 = pd.read_csv('./data/final_anime_1.csv')
final_anime_2 = pd.read_csv('./data/final_anime_2.csv')
final_anime_3 = pd.read_csv('./data/final_anime_3.csv')
final_anime_4 = pd.read_csv('./data/final_anime_4.csv')
final_anime_5 = pd.read_csv('./data/final_anime_5.csv')
final_profile = pd.read_csv('./data/final_profile.csv')
final_review = pd.read_csv('./data/final_review.csv')
final_watching_status = pd.read_csv('./data/final_watching_status.csv')

In [7]:
DFs = [final_anime_1, final_anime_2, final_anime_3, final_anime_4, final_anime_5, final_profile,
       final_review, final_watching_status]

for df in DFs:
    #df = strip(df)
    df = lowerCase(df)

## Source 1: Anime

In [8]:
final_anime_1 = final_anime_1.drop(labels=['ranked','popularity'], axis=1)
final_anime_1.rename(columns={'title': 'Name', 'uid':'anime1_ID'}, inplace=True)


In [9]:
final_anime_2 = final_anime_2.drop(labels=['ranked', 'popularity', 'score-10', 'score-9', 'score-8',
                                           'score-7', 'score-6', 'score-5', 'score-4', 'score-3',
                                           'score-2', 'score-1'], axis=1)

final_anime_2.rename(
    columns={'name': 'Name', 'mal_id': 'anime2_id'}, inplace=True)


In [10]:
Columns = list(set(final_anime_1.columns).intersection(final_anime_2.columns))

# Creating the anime table
anime = final_anime_1.merge(final_anime_2, on=['Name'], how='outer')

In [11]:
anime = remove(Columns, anime)

In [12]:
final_anime_3.rename(columns={'name':'Name','mal_id':'anime3_id'},inplace=True)
final_anime_3 = final_anime_3.drop_duplicates(subset=['Name'], keep='first')

Columns = list(set(final_anime_3.columns).intersection(anime.columns))
anime = anime.merge(final_anime_3, on='Name', how='outer')
anime = remove(Columns, anime)

In [13]:
final_anime_4 = final_anime_4.drop(labels=['score rank', 'popularity rank', 'air date', 'theme1',
                                           'theme2', 'theme3', 'theme4', 'theme5'], axis=1)
final_anime_4.rename(
    columns={'name': 'Name', 'num. of episodes': 'episodes',
             'demographic': 'demographic1'},inplace=True)


In [14]:
Columns = list(set(final_anime_4.columns).intersection(anime.columns))
anime = anime.merge(final_anime_4, on='Name', how='outer')
anime = remove(Columns, anime)

In [15]:
final_anime_5 = final_anime_5.drop(labels=['rank', 'minutes'], axis=1)
final_anime_5.rename(
    columns={'title': 'Name', 'rating':'score','year':'first_aired',}, inplace=True)

Columns = list(set(final_anime_5.columns).intersection(anime.columns))
anime = anime.merge(final_anime_5, on='Name', how='outer')
anime = remove(Columns, anime)

In [16]:
incase = anime.copy()

In [17]:
anime['animeID'] = anime.index + 1
anime['animeID'] = anime['animeID'].astype(str)
anime['animeID'] = 'anime_' + anime['animeID'] 

### Age

In [18]:
age_res = create_ID(anime,['age_restriction','meaning'],'ageID','age_')
anime = anime.merge(age_res, on=['age_restriction','meaning'],how='outer')
incase.shape[0] == anime.shape[0]

True

### Studio

In [19]:
anime['studio1'] = anime['studio1'].replace({'unknown': None})
anime['studio1'] = anime['studio1'].replace({'none': None})
studio = create_ID(anime, ['studio1'], 'studioID', 'st_')
anime = anime.merge(studio, on=['studio1'], how='outer')
incase.shape[0] == anime.shape[0]

True

### Demographic

In [20]:
(anime['demographic2'].isna().sum())/anime.shape[0]

0.951680905358926

The majority of the column *demographic2* is empty, as a result we ignore it.

In [21]:
demo = create_ID(anime,['demographic1'],'demoID','De_')
anime = anime.merge(demo, on=['demographic1'],how='outer')

### External links

In [22]:
links = create_ID(anime,['link','img_url'],'linkID','url_')
anime = anime.merge(links, on=['link','img_url'], how='outer')

### Popularity

In [24]:
popularity_list = ['members', 'score', 'completed', 'dropped',
                   'favorites', 'on-hold', 'plan to watch', 'watching']

popularity = create_ID(anime, popularity_list, 'popularityID', 'Po_')
Ws_list = ['completed','dropped','on-hold','plan to watch','watching']
watching_status = create_ID(popularity, Ws_list,'wsID','WS_')

In [25]:
popularity = popularity.merge(watching_status, on=Ws_list, how='outer')

In [26]:
anime = anime.merge(popularity, on=popularity_list, how='outer')

In [ ]:
popularity = popularity[['popularityID','members','score','wsID']]

In [27]:
final_anime_5['TopIMDb'] = True
imdb = final_anime_5[['Name', 'TopIMDb']]
anime = anime.merge(imdb, on='Name', how='outer')
anime['TopIMDb'] = anime['TopIMDb'].fillna(False)

### Genre

In [28]:
gList = ['genre1', 'genre2', 'genre3']
genre = create_ID(anime, gList, 'genreID', 'Ge_')
anime = anime.merge(genre, on=gList, how='outer')

## Review source

In [31]:
Review = final_review.copy()

In [32]:
Review['reviewID'] = Review.index + 1
Review['reviewID'] = Review['reviewID'].astype(str)
Review['reviewID'] = 'rev_' + Review['reviewID'] 

In [33]:
Review.to_csv('review_source.csv')

In [34]:
a = anime[['anime1_ID', 'animeID', 'linkID']]
a.rename(columns={'anime1_ID': 'anime_uid'}, inplace=True)
Review = Review.merge(a, on='anime_uid')
Review.rename(columns={'link': 'review_link'}, inplace=True)
Review = Review[['reviewID', 'profile', 'review_link', 'overall', 'linkID']]

## Profile source

In [36]:
final_profile['BD_year'] = pd.DatetimeIndex(final_profile['birthday']).year
final_profile['BD_month'] = pd.DatetimeIndex(final_profile['birthday']).month

In [37]:
difference = set(final_review['profile'].unique()) - set(final_profile['profile'].unique())

In [38]:
# All writers of review need to be in profile
for diff in difference:
    new_row = {'profile':diff}
    final_profile = final_profile.append(new_row, ignore_index=True)

In [40]:
final_profile.to_csv('final_profile.csv')

In [41]:
Reviewer = final_profile.copy()

In [42]:
Rlist = ['profile', 'gender', 'birthday', 'favanime1',
         'favanime2', 'favanime3', 'favanime4', 'favanime5']
Reviewer = create_ID(Reviewer, Rlist, 'ReviewerID', 'Reviewer_')


In [ ]:
#Reviewer.to_csv('Reviewer.csv')

## Date source

In [43]:
anime['year'] = pd.DatetimeIndex(anime['first_aired']).year
anime['month'] = pd.DatetimeIndex(anime['first_aired']).month
date = create_ID(anime,['year','month'],'dateID','date_')

In [44]:
Reviewer['year'] = pd.DatetimeIndex(Reviewer['birthday']).year
Reviewer['month'] = pd.DatetimeIndex(Reviewer['birthday']).month

dateR = create_ID(Reviewer, ['year', 'month'], 'dateID', 'date_')
dateR['year'] = dateR['year'].astype(int)
dateR['month'] = dateR['month'].astype(int)


In [45]:
date = date.merge(dateR, on=['year','month'],how='outer')
date['year'] = date['year'].astype(int)
date['month'] = date['month'].astype(int)

In [46]:
date = date.sort_values(['year','month']).reset_index()
date = date.drop(labels=['index','dateID_x','dateID_y'], axis=1)

In [47]:
date['dateID'] = date.index + 1
date['dateID'] = date['dateID'].astype(str)
date['dateID'] = 'date_' + date['dateID'] 

In [48]:
date.to_csv('date_source.csv')

In [52]:
anime['score'] = anime['score'].replace({'unknown': np.nan})
anime['score'] = anime['score'].astype(float)

In [54]:
for c in list(anime.columns):
    if anime[c].dtype == 'object':
        anime[c] = anime[c].replace({'unknown': None})
        anime[c] = anime[c].replace({'none': None})

In [55]:
anime.to_csv('anime_source.csv')